# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,whitehorse,60.7161,-135.0538,-1.57,86,75,6.17,CA,1701895533
1,1,edinburgh of the seven seas,-37.0676,-12.3116,14.35,76,10,8.10,SH,1701895534
2,2,bethel,41.3712,-73.4140,3.62,71,100,2.57,US,1701895518
3,3,nadi,-17.8000,177.4167,28.00,69,20,2.06,FJ,1701895534
4,4,klyuchi,52.2667,79.1667,-12.12,68,100,9.04,RU,1701895534


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, tiles= True, color='City',
                               
                            size='Humidity')

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
Humid = city_data_df["Humidity"]<15
Humid_df= city_data_df.loc[Humid]
# Drop any rows with null values
new_city_data_df=Humid_df.dropna()

# Display sample data
new_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
102,102,san rafael,-34.6177,-68.3301,30.09,7,1,3.93,AR,1701895547
107,107,moussoro,13.6408,16.4901,24.63,10,0,5.02,TD,1701895548
126,126,kidal,18.4411,1.4078,24.97,12,0,2.90,ML,1701895550
189,189,taoudenni,22.6783,-3.9836,22.27,11,100,2.01,ML,1701895554
214,214,menaka,15.9182,2.4022,26.53,13,0,3.99,ML,1701895556
305,305,calingasta,-31.3308,-69.4078,29.05,4,0,7.32,AR,1701895574
361,361,canon city,38.4410,-105.2424,23.09,10,0,7.20,US,1701895584
401,401,niafunke,15.9322,-3.9906,27.75,12,1,4.25,ML,1701895591


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= new_city_data_df[["City","Country","Lat","Lng","Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
102,san rafael,AR,-34.6177,-68.3301,7,
107,moussoro,TD,13.6408,16.4901,10,
126,kidal,ML,18.4411,1.4078,12,
189,taoudenni,ML,22.6783,-3.9836,11,
214,menaka,ML,15.9182,2.4022,13,
305,calingasta,AR,-31.3308,-69.4078,4,
361,canon city,US,38.4410,-105.2424,10,
401,niafunke,ML,15.9322,-3.9906,12,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius =  10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    print(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Hotel Regional', 'country': 'Argentina', 'country_code': 'ar', 'state': 'Mendoza', 'county': 'Departamento San Rafael', 'city': 'Distrito Ciudad de San Rafael', 'postcode': '5602', 'district': 'San Rafael. Cañada Seca. Atuel Norte', 'street': 'Avenida Bartolomé Mitre', 'lon': -68.3294781, 'lat': -34.6165402, 'state_code': 'M', 'formatted': 'Hotel Regional, Avenida Bartolomé Mitre, San Rafael. Cañada Seca. Atuel Norte, 5602 Distrito Ciudad de San Rafael, Argentina', 'address_line1': 'Hotel Regional', 'address_line2': 'Avenida Bartolomé Mitre, San Rafael. Cañada Seca. Atuel Norte, 5602 Distrito Ciudad de San Rafael, Argentina', 'categories': ['accommodation', 'accommodation.hotel'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': 

{'type': 'FeatureCollection', 'features': []}
moussoro - nearest hotel: No hotel found
{'type': 'FeatureCollection', 'features': []}
kidal - nearest hotel: No hotel found
{'type': 'FeatureCollection', 'features': []}
taoudenni - nearest hotel: No hotel found
{'type': 'FeatureCollection', 'features': []}
menaka - nearest hotel: No hotel found
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Nora Hôtel', 'country': 'Argentina', 'country_code': 'ar', 'state': 'San Juan', 'county': 'Calingasta', 'city': 'Calingasta', 'postcode': 'J5403', 'street': 'Aldo Cantoni', 'lon': -69.4202774, 'lat': -31.3348069, 'state_code': 'J', 'formatted': 'Nora Hôtel, Aldo Cantoni, J5403 Calingasta, Argentina', 'address_line1': 'Nora Hôtel', 'address_line2': 'Aldo Cantoni, J5403 Calingasta, Argentina', 'categories': ['accommodation', 'accommodation.hotel', 'internet_access', 'internet_access.free'], 'details': ['details.facilities'], 'datasource': {'sourcename': 'openstreet

,City,Country,Lat,Lng,Humidity,Hotel Name
102,san rafael,AR,-34.6177,-68.3301,7,Hotel Regional
107,moussoro,TD,13.6408,16.4901,10,No hotel found
126,kidal,ML,18.4411,1.4078,12,No hotel found
189,taoudenni,ML,22.6783,-3.9836,11,No hotel found
214,menaka,ML,15.9182,2.4022,13,No hotel found
305,calingasta,AR,-31.3308,-69.4078,4,Nora Hôtel
361,canon city,US,38.4410,-105.2424,10,No hotel found
401,niafunke,ML,15.9322,-3.9906,12,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot

hotel_map = hotel_df.hvplot.points(

'Lng',
'Lat',
geo = True,
title = 'OMS',
hover_cols = ["Hotel Name", "Country"]

)

# Display the map
hotel_map

:Points   [Lng,Lat]   (Hotel Name,Country)